<a href="https://colab.research.google.com/github/Gail529/chess_ML/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold


  

In [3]:
data=pd.read_csv('/content/games.csv')


In [4]:
data.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [5]:
data.columns
del data['id']
del data['created_at']
del data['last_move_at']
del data['white_id']
del data['black_id']
del data['opening_name']
del data['opening_eco']
del data['moves']


In [6]:
data.columns


Index(['rated', 'turns', 'victory_status', 'winner', 'increment_code',
       'white_rating', 'black_rating', 'opening_ply'],
      dtype='object')

Encoding Categorical Variables


In [7]:

# Categorical boolean mask
categorical_feature_mask = data.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = data.columns[categorical_feature_mask].tolist()

# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
data[categorical_cols].head(10)


,victory_status,winner,increment_code
0,2,2,110
1,3,0,298
2,1,2,298
3,1,2,187
4,1,2,247
5,0,1,19
6,3,2,19
7,3,0,114
8,3,0,102
9,1,2,19


In [8]:
cols=['rated', 'turns', 'victory_status', 'increment_code','white_rating', 'black_rating', 'opening_ply']
x=data[cols]
y=data['winner']
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.20,random_state = 42)
                                                 

# Using Grid Search CV to find the best parameters

Logistic Regression


In [ ]:

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train,y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


K Nearest Neighbours

In [8]:
knn_grid =GridSearchCV(estimator=KNeighborsClassifier() ,param_grid={'n_neighbors':np.arange(1,20)} ,cv=5)
knn_grid.fit(x,y)
knn_grid.best_params_


{'n_neighbors': 19}

Support Vector Machines


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.1,1,10],'gamma':[1,0.1,0.01],'kernel':['linear']}
scores ='accuracy'
grid = GridSearchCV(SVC(), param_grid,n_jobs=-1,scoring='accuracy')
 
def fit_cv_subsample (pipe_cv, X, y, n_max = 5000):
    '''
    This function fits a CV in a subsample of the first n_max rows
    returns the trained pipe and the best estimator
    '''
    X_sub = X[0:n_max]
    y_sub = y[0:n_max]
    pipe_cv.fit(X_sub,y_sub)
    #pipe_cv.best_estimator_.fit(X,y)
    return pipe_cv, pipe_cv.best_estimator_


results, best_model = fit_cv_subsample(grid,X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.1,1,10],'gamma':[1,0.1,0.01],'kernel':['linear']}
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

grid = GridSearchCV(SVC(), param_grid, scoring='%s_macro' % score)
grid.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(grid.best_params_)


# Tuning hyper-parameters for precision

# Tuning hyper-parameters for recall



In [ ]:
from sklearn import model_selection,svm
SVM=svm.SVC(C=1.0,kernel='linear')
SVM.fit(X_train,y_train)

In [ ]:
predictions=SVM.predict(x_tfidf_test)
from sklearn.metrics import classification_report  #training the model using tfidf provides a higher accuracy for SVM
print(classification_report(y_test,predictions))

In [ ]:
print(grid.best_estimator_)

# Hard Voting Ensemble 

In [ ]:
models = [] 
models.append(('LR',LogisticRegression()))
models.append(('SVC', SVC(gamma ='auto', probability = True))) 
models.append(('DTC', DecisionTreeClassifier()))
models.append(('knn1', KNeighborsClassifier(n_neighbors=1)))
  

In [ ]:
vot_hard = VotingClassifier(estimators = models, voting ='hard') 
vot_hard.fit(X_train, y_train) 
y_pred = vot_hard.predict(X_test) 
  
# using accuracy_score metric to predict accuracy 
score = accuracy_score(y_test, y_pred) 
print("Hard Voting Score % d" % score) 
  
# Voting Classifier with soft voting 
vot_soft = VotingClassifier(estimators = models, voting ='soft') 
vot_soft.fit(X_train, y_train) 
y_pred = vot_soft.predict(X_test) 
  
# using accuracy_score 
score = accuracy_score(y_test, y_pred) 
print("Soft Voting Score % d" % score)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Hard Voting Score  0


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
